In [53]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute, QuantumRegister, ClassicalRegister
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel

In [80]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [54]:
rc = random_circuit(2, 4, 2, measure=True)

In [55]:
rc.draw(fold=-1)

┌────────────────────────┐┌──────────────────────────┐┌────────────┐┌─┐   
q_0: ─■────────────┤1                       ├┤ U3(1.3405,4.6647,2.6013) ├┤ Rx(3.8721) ├┤M├───
      │U1(0.92579) │  {XX-YY}(5.4309,4.269) │├──────────────────────────┤├────────────┤└╥┘┌─┐
q_1: ─■────────────┤0                       ├┤ U3(3.5748,1.1242,5.0772) ├┤ Ry(2.6901) ├─╫─┤M├
                   └────────────────────────┘└──────────────────────────┘└────────────┘ ║ └╥┘
c: 2/═══════════════════════════════════════════════════════════════════════════════════╩══╩═
                                                                                        0  1

In [56]:
qr = QuantumRegister(5)
larger_qc = QuantumCircuit(qr)

In [57]:
larger_qc.append(rc, qr[:2])

In [58]:
larger_qc.draw()

┌──────────────┐
q5_0: ┤0             ├
      │  circuit-125 │
q5_1: ┤1             ├
      └──────────────┘
q5_2: ────────────────
                      
q5_3: ────────────────
                      
q5_4: ────────────────

In [59]:
rc2 = random_circuit(2, 4, 2, measure=True)

In [63]:
rc = rc.compose(rc2)

In [64]:
rc.draw(fold=-1)

┌────────────────────────┐┌──────────────────────────┐┌────────────┐┌─┐┌───────────┐┌──────────────────────────┐     ┌──────┐    ┌────┐    ┌─┐   
q_0: ─■────────────┤1                       ├┤ U3(1.3405,4.6647,2.6013) ├┤ Rx(3.8721) ├┤M├┤ P(2.3801) ├┤ U3(0.71851,4.2325,6.022) ├─────┤1     ├────┤ √X ├────┤M├───
      │U1(0.92579) │  {XX-YY}(5.4309,4.269) │├──────────────────────────┤├────────────┤└╥┘└────┬─┬────┘└──────────┬───┬───────────┘┌───┐│  Dcx │┌───┴────┴───┐└╥┘┌─┐
q_1: ─■────────────┤0                       ├┤ U3(3.5748,1.1242,5.0772) ├┤ Ry(2.6901) ├─╫──────┤M├────────────────┤ T ├────────────┤ Y ├┤0     ├┤ Rx(6.2509) ├─╫─┤M├
                   └────────────────────────┘└──────────────────────────┘└────────────┘ ║      └╥┘                └───┘            └───┘└──────┘└────────────┘ ║ └╥┘
c: 2/═══════════════════════════════════════════════════════════════════════════════════╩═══════╩══════════════════════════════════════════════════════════════╩══╩═
                                                                                        0       1                                                              0  1

In [89]:
def construct_tiling_rc(qbs_with_gates, num_qubit, depth):
    assert len(qbs_with_gates) <= num_qubit
    rc = random_circuit(len(qbs_with_gates), depth, 2, measure=False)
    cr = ClassicalRegister(len(qbs_with_gates))
    qr = QuantumRegister(num_qubit)
    qc = QuantumCircuit(qr, cr)
    qc.append(rc, [qr[i] for i in qbs_with_gates])
    qc.measure(qbs_with_gates, cr)
    return qc

transpile(construct_tiling_rc([0, 1, 4], 5, 5), backend=backend_ideal).draw(fold=-1)

┌───┐                                 ┌──────────────┐     ┌────────────┐                                                         ┌─┐      
q73_0: ───────────────────────────┤ Y ├─────────────■───────────────────┤1             ├─────┤ Ry(0.6873) ├────────────────────────────────────────────■────────────┤M├──────
       ┌─────────────────────────┐└─┬─┘┌────────────┴────────────┐┌────┐│              │     └────────────┘                                            │P(6.2509)   └╥┘┌─┐   
q73_1: ┤ U(2.8378,1.9192,5.1095) ├──┼──┤ U3(2.302,3.8921,3.1311) ├┤ √X ├┤              ├──────────────────────────■─────────────────────────────■──────■─────────────╫─┤M├───
       └─────────────────────────┘  │  └─────────────────────────┘└────┘│              │                          │                             │                    ║ └╥┘   
q73_2: ─────────────────────────────┼───────────────────────────────────┤  Ryy(3.3692) ├──────────────────────────┼─────────────────────────────┼────────────────────╫──╫────
                                    │                                   │              │                          │                             │                    ║  ║    
q73_3: ─────────────────────────────┼───────────────────────────────────┤              ├──────────────────────────┼─────────────────────────────┼────────────────────╫──╫────
                                    │            ┌──────┐               │              │┌──────────────────────┐┌─┴─┐┌───────────────────────┐┌─┴─┐┌───────────────┐ ║  ║ ┌─┐
q73_4: ─────────────────────────────■────────────┤ √Xdg ├───────────────┤0             ├┤ Ry(1.13582778076033) ├┤ X ├┤ Ry(-1.13582778076032) ├┤ X ├┤ Rx(0.0041606) ├─╫──╫─┤M├
                                                 └──────┘               └──────────────┘└──────────────────────┘└───┘└───────────────────────┘└───┘└───────────────┘ ║  ║ └╥┘
 c4: 3/══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩═
                                                                                                                                                                     0  1  2

TypeError: 'NoneType' object is not iterable